In [1]:
import cv2
import mediapipe as mp

def extract_eye_mouth(image):
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(image_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_eye_pts = []
            right_eye_pts = []
            mouth_pts = []

            for i in range(36, 42):  # Landmark indices for left eye
                x = int(face_landmarks.landmark[i].x * image.shape[1])
                y = int(face_landmarks.landmark[i].y * image.shape[0])
                left_eye_pts.append((x, y))

            for i in range(42, 48):  # Landmark indices for right eye
                x = int(face_landmarks.landmark[i].x * image.shape[1])
                y = int(face_landmarks.landmark[i].y * image.shape[0])
                right_eye_pts.append((x, y))

            for i in range(48, 68):  # Landmark indices for mouth
                x = int(face_landmarks.landmark[i].x * image.shape[1])
                y = int(face_landmarks.landmark[i].y * image.shape[0])
                mouth_pts.append((x, y))

            return left_eye_pts, right_eye_pts, mouth_pts

    return None, None, None




# Set up video capture
cap = cv2.VideoCapture(0)  # Use 0 for the default camera

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

while True:
    ret, frame = cap.read()  # Read frame from video capture
    
    # Flip frame horizontally for mirror effect
    frame = cv2.flip(frame, 1)

    # Extract eye and mouth regions
    left_eye_pts, right_eye_pts, mouth_pts = extract_eye_mouth(frame)

    # Draw eye and mouth regions on the frame
    if left_eye_pts and right_eye_pts and mouth_pts:
        for pt in left_eye_pts:
            cv2.circle(frame, pt, 2, (0, 255, 0), -1)
        
        for pt in right_eye_pts:
            cv2.circle(frame, pt, 2, (0, 255, 0), -1)
        
        for pt in mouth_pts:
            cv2.circle(frame, pt, 2, (0, 255, 0), -1)

    # Display the frame
    cv2.imshow('Video', frame)

    # Exit loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()